In [1]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
from urllib.parse import urlparse, quote
import folium
import pandas as pd
from glob import glob

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.get('http://www.opinet.co.kr/searRgSelect.do')
time.sleep(2)

driver.find_element_by_css_selector('.ic_m1').click()
time.sleep(2)
driver.find_element_by_xpath('//*[@id="SEARCH_MOD4"]').click()
time.sleep(1)

station_list = ['우장산주유소', '직영관문주유소', '화곡현대주유소', '신화곡주유소', '경인고속주유소', '목화주유소', '뉴신정주유소', '화곡역주유소', '마곡대성주유소', '방화대성주유소']
gasoline_list = []; diesel_list = []; addr_list=[]; tel_list=[]
for station in station_list:
    search = driver.find_element_by_xpath('//*[@id="OS_NM"]')
    time.sleep(1)
    search.clear()
    time.sleep(2)
    search.send_keys(station)
    time.sleep(1)
    search.submit()
    time.sleep(1)
    driver.find_element_by_xpath('//*[@id="body1"]/tr/td[1]/a').click()
    map_move = driver.find_element_by_xpath('//*[@id="map_div"]/div[4]/div[2]/div[2]/div/div[3]').click()
    time.sleep(1)
    tel = driver.find_element_by_xpath('//*[@id="phn_no"]').text
    addr = driver.find_element_by_xpath('//*[@id="rd_addr"]').text
    gasoline = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[2]').text
    diesel = driver.find_element_by_xpath('//*[@id="body1"]/tr/td[3]/font').text
    time.sleep(10)

    addr_list.append(addr)
    tel_list.append(tel)
    gasoline_list.append(int(gasoline))
    diesel_list.append(int(diesel))

driver.quit()

In [3]:
addr_list

['서울 강서구 강서로 292 (내발산동)',
 '서울 강서구 공항대로 314',
 '서울 강서구 강서로 151 (화곡동)',
 '서울 강서구 국회대로 71',
 '서울 강서구 국회대로 225 (화곡동)',
 '서울 강서구 국회대로 251 (화곡동)',
 '서울 강서구 곰달래로 207(화곡동)',
 '서울특별시 강서구 강서로 154 (화곡동)',
 '서울 강서구 강서로 457',
 '서울 강서구 양천로 176']

In [4]:
tel_list

['02-3664-7163',
 '02-3665-2051',
 '02-2607-1942',
 '02-2611-3161',
 '02-2608-5151',
 '02-2654-4564',
 '02-2601-0077',
 '02-2691-5151',
 '02-3664-7781',
 '02-3663-7781']

In [5]:
df = pd.DataFrame({
    'station_name':station_list,
    'address':addr_list,
    'tel':tel_list,
    'gasoline':gasoline_list,
    'diesel':diesel_list
})
df

,station_name,address,tel,gasoline,diesel
0,우장산주유소,서울 강서구 강서로 292 (내발산동),02-3664-7163,1358,1187
1,직영관문주유소,서울 강서구 공항대로 314,02-3665-2051,1358,1187
2,화곡현대주유소,서울 강서구 강서로 151 (화곡동),02-2607-1942,1326,1126
3,신화곡주유소,서울 강서구 국회대로 71,02-2611-3161,1315,1115
4,경인고속주유소,서울 강서구 국회대로 225 (화곡동),02-2608-5151,1266,1072
5,목화주유소,서울 강서구 국회대로 251 (화곡동),02-2654-4564,1266,1072
6,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),02-2601-0077,1243,1043
7,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),02-2691-5151,1243,1043
8,마곡대성주유소,서울 강서구 강서로 457,02-3664-7781,1365,1185
9,방화대성주유소,서울 강서구 양천로 176,02-3663-7781,1365,1175


In [6]:
key_fd = open('kakaomapkey.txt', mode='r')
kmap_key = key_fd.read(100)
key_fd.close()

In [7]:
lat_list = []; lng_list = []
for i in df.index:
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + quote(df['address'][i])
    result = requests.get(url,
            headers={"Authorization": "KakaoAK "+kmap_key}).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
    lat_list.append(lat)
    lng_list.append(lng)

In [8]:
df['lat'] = lat_list; df['lng'] = lng_list
df

,station_name,address,tel,gasoline,diesel,lat,lng
0,우장산주유소,서울 강서구 강서로 292 (내발산동),02-3664-7163,1358,1187,37.551803,126.836681
1,직영관문주유소,서울 강서구 공항대로 314,02-3665-2051,1358,1187,37.557842,126.842395
2,화곡현대주유소,서울 강서구 강서로 151 (화곡동),02-2607-1942,1326,1126,37.540242,126.841164
3,신화곡주유소,서울 강서구 국회대로 71,02-2611-3161,1315,1115,37.527320,126.842879
4,경인고속주유소,서울 강서구 국회대로 225 (화곡동),02-2608-5151,1266,1072,37.529645,126.859835
5,목화주유소,서울 강서구 국회대로 251 (화곡동),02-2654-4564,1266,1072,37.530055,126.862834
6,뉴신정주유소,서울 강서구 곰달래로 207(화곡동),02-2601-0077,1243,1043,37.531562,126.856035
7,화곡역주유소,서울특별시 강서구 강서로 154 (화곡동),02-2691-5151,1243,1043,37.540768,126.841472
8,마곡대성주유소,서울 강서구 강서로 457,02-3664-7781,1365,1185,37.566243,126.840421
9,방화대성주유소,서울 강서구 양천로 176,02-3663-7781,1365,1175,37.572898,126.823440


In [9]:
map = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=13)

for i in df.index:
    frame = f"{df['station_name'][i]}"
    popup = folium.Popup(frame, max_width=130)
    if df['gasoline'][i] == min(df['gasoline']):
        folium.Marker(
            location = [df['lat'][i], df['lng'][i]],
            tooltip=(f"휘발유:{df['gasoline'][i]}원, 가솔린:{df['diesel'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='red', icon='thumbs-up')
            ).add_to(map)
    else:
        folium.Marker(
            location = [df['lat'][i], df['lng'][i]],
            tooltip=(f"휘발유:{df['gasoline'][i]}원, 가솔린:{df['diesel'][i]}원"),
            popup=popup,
            icon=folium.Icon(color='blue', icon='tint')
            ).add_to(map)
map